In [1]:
from numpy import random
import numpy 
import re
import os
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import shutil
from time import sleep
import pandas as pd
from matplotlib.ticker import FormatStrFormatter
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from random import randint, randrange
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, f1_score

In [2]:

#The first part of the project was of course, the Scraping and Crawling. By looking at our base URL
#(I opted for https://www.billboard.com/charts/hot-100/2021-12-12/)
#we can see several things:
#1. The list is easily seperated into rows that are detectable via BeutifulSoup
#2. The parts that aren't just given as numbers are given as unique images, such as wether a song went up or down
#3. We can use the URL of the image to decipher whether or it went up, down, stayed the same or is new entirely
#4. The site is very friendly to crawling, as the date field is conveniant and forgiving (if you don't use the first day of a week given, it'll just display the results
#   for that week anyway)
#With that information, we can start a basic scraper and crawler
def data_fetch(URL, amount_of_weeks):
    weeks_done = 0
    song_list = []
    while weeks_done < amount_of_weeks:
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        mydivs = soup.findAll("div", class_="o-chart-results-list-row-container")
        currentYear = URL[41:45]
        currentMonth = URL[46:48]
        currentDay = URL[49:51]
        i = 1
        for div in mydivs:
            song_name = div.find("h3", id="title-of-a-story")
            details = div.find("ul",
                               class_="lrv-a-unstyle-list lrv-u-flex lrv-u-height-100p lrv-u-flex-direction-row u-background-color-grey-lightest@mobile-max")
            details = details.findAll("span")
            lastWeek = details[0].text[1:-1]
            peak_pos = details[1].text[1:-1]
            weeksOnChart = details[2].text[1:-1]
            if "-" in lastWeek:
                if weeksOnChart == "1":
                    up_down = "New"
                else:
                    up_down = "Re-Entry"
            elif i > int(lastWeek):
                up_down = "Up"
            elif i < int(lastWeek):
                up_down = "Down"
            elif i == int(lastWeek):
                up_down = "Same"
            artist_name = div.findAll("span")[1].text[1:-1]
            if artist_name == "NEW" or artist_name == "RE-\nENTRY":
                artist_name = div.findAll("span")[3].text[1:-1]
            song_list.append([song_name.text[1:-1], artist_name, i, up_down, lastWeek, peak_pos, weeksOnChart,currentYear+"-"+currentMonth+"-"+currentDay])
            i += 1

        #the following few lines all have to do with going back weeks, making sure that we switch days/months/years in a proper fashion
        if int(currentDay) - 7 < 1:
            currentMonth = str(int(currentMonth) - 1)
            print(currentMonth)
            currentDay = str(31 + int(currentDay))  # while this dosen't differntiate different month lengths, the billboard website treats all days within a given week as the same so at worst this will create a duplicative that will be disposed of later
        currentDay = str(int(currentDay) - 7)
        weeks_done += 1
        if len(currentDay) == 1:
            currentDay = "0" + currentDay
        if len(currentMonth) == 1:
            currentMonth = "0" + currentMonth
        if int(currentMonth) < 1:
            currentMonth = "12"
            currentYear = str(int(currentYear) - 1)
        URL = URL[:-11] + currentYear + "-" + currentMonth + "-" + currentDay + "/"
        print(URL)
        sleep(1)
    myDB = pd.DataFrame(song_list,
                        columns=["Song_Name", "Artist_Name", "Current_Position", "Relative_Movement", "Last_Week",
                                 "Peak_Position", "Weeks_OnChart","Current_Date"])
    return myDB

In [3]:
#example for 3 pages:
myDB=data_fetch("https://www.billboard.com/charts/hot-100/2021-12-12/",3)
myDB

https://www.billboard.com/charts/hot-100/2021-12-05/
11
https://www.billboard.com/charts/hot-100/2021-11-29/
https://www.billboard.com/charts/hot-100/2021-11-22/


,Song_Name,Artist_Name,Current_Position,Relative_Movement,Last_Week,Peak_Position,Weeks_OnChart,Current_Date
0,Easy On Me,Adele,1,Same,1,1,9,2021-12-12
1,All I Want For Christmas Is You,Mariah Carey,2,Down,3,1,48,2021-12-12
2,Rockin' Around The Christmas Tree,Brenda Lee,3,Down,4,2,42,2021-12-12
3,Stay,The Kid LAROI & Justin Bieber,4,Up,2,1,22,2021-12-12
4,A Holly Jolly Christmas,Burl Ives,5,Down,7,4,23,2021-12-12
...,...,...,...,...,...,...,...,...
295,Too Easy,Gunna & Future,96,Re-Entry,-,38,8,2021-11-29
296,Escape Plan,Travis Scott,97,Up,71,11,3,2021-11-29
297,Who's In Your Head,Jonas Brothers,98,Re-Entry,-,92,5,2021-11-29
298,Jugaste y Sufri,Eslabon Armado Featuring DannyLux,99,Re-Entry,-,69,7,2021-11-29
